In [1]:
import numpy as np
import pandas as pd

In [2]:
from sklearn.linear_model import LogisticRegression

In [3]:
#ignoring the warnings
import sys
if not sys.warnoptions:
    import warnings
    warnings.simplefilter("ignore")

In [4]:
def clean(df):
    df['income']=df['ApplicantIncome']+df['CoapplicantIncome']
    df['loanpm']=df['LoanAmount']+df['Loan_Amount_Term']
    df.drop(['LoanAmount','Loan_ID','Loan_Amount_Term','ApplicantIncome','CoapplicantIncome'],axis='columns',inplace=True)
    df['Gender']=df['Gender'].replace({'Male':1,'Female':0})
    df['Married']=df['Married'].replace({'Yes':1,'No':0})
    df['Education']=df['Education'].replace({'Graduate':1,'Not Graduate':0})
    df['Self_Employed']=df['Self_Employed'].replace({'Yes':1,'No':0})
    df['Dependents']=df['Dependents'].replace({'3+':3,'1':1,'2':2})
    df['Property_Area']=df['Property_Area'].replace({'Urban':3,'Rural':1,'Semiurban':2})
    df['income_bins']=pd.cut(df.income,np.linspace(1400,81000,dtype='int',num=20),labels=np.linspace(0,18,dtype='int',num=19))
    df['Income']=pd.cut(df.income_bins,[-1,0,1,2,3,18],labels=[0,1,2,3,4])
    df['loan']=pd.cut(df.loanpm,np.linspace(100,1000,dtype='int',num=10),labels=[0,1,2,3,4,5,6,7,8])
    df.drop(['income','loanpm','income_bins'],axis=1,inplace=True)
    df=df.dropna()
    df.Dependents=df.Dependents.astype('int')
    return df

In [5]:
#filling the unknown values of columns rather than droping
def get_filled(train1,col):
    train1=train.loc[train[col].isnull()]
    train1[col]='0'
    print(train1.isnull().sum())
    return train1

In [6]:
def new_df(df1,df2,col,df3):
    df_new=pd.concat([df1,df2],axis=0)
    df_new.drop('Loan_Status',axis=1,inplace=True)
    df3.drop('Loan_Status',axis=1,inplace=True)
    df_new=clean(df_new)
    x=df_new.drop(col,axis=1)
    y=df_new[col]
    y=y.astype('int')
    y.columns=[col]
    lreg=LogisticRegression()
    lreg.fit(x,y)
    df3[col]=lreg.predict(df3.drop(col,axis=1))
    a=df2.loc[df2[col].notnull(),col]
    b=df3[col]
    c=pd.concat([a,b],axis=0)
    c.sort_index(axis=0,ascending=True,inplace=True)
    df2.drop(col,axis=1,inplace=True)
    df2=pd.concat([df2,c],axis=1)
    return df2
    #df1==df(cleaned)
    #df2==train(cleaned wrt col)
    #df3==train(cleaned)

In [7]:
df=pd.read_csv('test.csv')
df.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area
0,LP001015,Male,Yes,0,Graduate,No,5720,0,110.0,360.0,1.0,Urban
1,LP001022,Male,Yes,1,Graduate,No,3076,1500,126.0,360.0,1.0,Urban
2,LP001031,Male,Yes,2,Graduate,No,5000,1800,208.0,360.0,1.0,Urban
3,LP001035,Male,Yes,2,Graduate,No,2340,2546,100.0,360.0,NaN,Urban
4,LP001051,Male,No,0,Not Graduate,No,3276,0,78.0,360.0,1.0,Urban


In [8]:
train0=pd.read_csv('train.csv')
train=train0.copy()
train.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


In [9]:
train.isnull().sum()

Loan_ID               0
Gender               13
Married               3
Dependents           15
Education             0
Self_Employed        32
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount           22
Loan_Amount_Term     14
Credit_History       50
Property_Area         0
Loan_Status           0
dtype: int64

# filling the unknown values of gender

In [10]:
train1=get_filled(train,'Gender')

Loan_ID              0
Gender               0
Married              0
Dependents           0
Education            0
Self_Employed        0
ApplicantIncome      0
CoapplicantIncome    0
LoanAmount           0
Loan_Amount_Term     0
Credit_History       1
Property_Area        0
Loan_Status          0
dtype: int64


In [11]:
train1.Credit_History.replace({np.nan:0},inplace=True)
train1=clean(train1)
train1.isnull().sum()

Gender            0
Married           0
Dependents        0
Education         0
Self_Employed     0
Credit_History    0
Property_Area     0
Loan_Status       0
Income            0
loan              0
dtype: int64

In [12]:
train['Gender']=train['Gender'].replace({'Male':'1','Female':'0'})
train=new_df(df,train,'Gender',train1)
train.isnull().sum()

Loan_ID               0
Married               3
Dependents           15
Education             0
Self_Employed        32
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount           22
Loan_Amount_Term     14
Credit_History       50
Property_Area         0
Loan_Status           0
Gender                0
dtype: int64

# filling the unknown values of dependents

In [13]:
train1=get_filled(train,'Dependents')

Loan_ID              0
Married              3
Dependents           0
Education            0
Self_Employed        0
ApplicantIncome      0
CoapplicantIncome    0
LoanAmount           2
Loan_Amount_Term     1
Credit_History       0
Property_Area        0
Loan_Status          0
Gender               0
dtype: int64


In [14]:
train1.Loan_Amount_Term.median()

360.0

In [15]:
train1.LoanAmount.median()

138.0

In [16]:
train1.Married.mode()

0    Yes
dtype: object

In [17]:
train1.Loan_Amount_Term.replace({np.nan:360},inplace=True)
train1.LoanAmount.replace({np.nan:138},inplace=True)
train1.Married.replace({np.nan:'Yes'},inplace=True)
train1['Gender']=train1['Gender'].replace({1:'Male',0:'Female'})

In [18]:
train1=clean(train1)
train1.isnull().sum()

Married           0
Dependents        0
Education         0
Self_Employed     0
Credit_History    0
Property_Area     0
Loan_Status       0
Gender            0
Income            0
loan              0
dtype: int64

In [19]:
train['Dependents']=train['Dependents'].replace({'3+':3,'1':1,'2':2})
train=new_df(df,train,'Dependents',train1)
train.isnull().sum()

Loan_ID               0
Married               3
Education             0
Self_Employed        32
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount           22
Loan_Amount_Term     14
Credit_History       50
Property_Area         0
Loan_Status           0
Gender                0
Dependents            0
dtype: int64

# filling the unknown values of self_employed

In [20]:
train1=get_filled(train,'Self_Employed')

Loan_ID              0
Married              0
Education            0
Self_Employed        0
ApplicantIncome      0
CoapplicantIncome    0
LoanAmount           1
Loan_Amount_Term     1
Credit_History       6
Property_Area        0
Loan_Status          0
Gender               0
Dependents           0
dtype: int64


In [21]:
train.Credit_History.mode()

0    1.0
dtype: float64

In [22]:
train1.LoanAmount.median()

125.0

In [23]:
train1.Loan_Amount_Term.median()

360.0

In [24]:
train1.Credit_History.replace({np.nan:1},inplace=True)
train1.Loan_Amount_Term.replace({np.nan:360},inplace=True)
train1.LoanAmount.replace({np.nan:125},inplace=True)
train1['Gender']=train1['Gender'].replace({1:'Male',0:'Female'})
train1['Dependents']=train1['Dependents'].replace({'3+':3,'1':1,'2':2})
train1=clean(train1)
train1.isnull().sum()

Married           0
Education         0
Self_Employed     0
Credit_History    0
Property_Area     0
Loan_Status       0
Gender            0
Dependents        0
Income            0
loan              0
dtype: int64

In [25]:
train['Self_Employed']=train['Self_Employed'].replace({'Yes':1,'No':0})
train=new_df(df,train,'Self_Employed',train1)
train.isnull().sum()

Loan_ID               0
Married               3
Education             0
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount           22
Loan_Amount_Term     14
Credit_History       50
Property_Area         0
Loan_Status           0
Gender                0
Dependents            0
Self_Employed         0
dtype: int64

# filling the unknown values of credit_history

In [26]:
train1=get_filled(train,'Credit_History')

Loan_ID              0
Married              0
Education            0
ApplicantIncome      0
CoapplicantIncome    0
LoanAmount           1
Loan_Amount_Term     0
Credit_History       0
Property_Area        0
Loan_Status          0
Gender               0
Dependents           0
Self_Employed        0
dtype: int64


In [27]:
train1.LoanAmount.median()

130.0

In [28]:
train1.LoanAmount.replace({np.nan:130},inplace=True)
train1['Gender']=train1['Gender'].replace({1:'Male',0:'Female'})
train1['Dependents']=train1['Dependents'].replace({'3+':3,'1':1,'2':2})
train1['Self_Employed']=train1['Self_Employed'].replace({1:'Yes',0:'No'})
train1=clean(train1)
train1.isnull().sum()

Married           0
Education         0
Credit_History    0
Property_Area     0
Loan_Status       0
Gender            0
Dependents        0
Self_Employed     0
Income            0
loan              0
dtype: int64

In [29]:
train=new_df(df,train,'Credit_History',train1)
train.isnull().sum()

Loan_ID               0
Married               3
Education             0
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount           22
Loan_Amount_Term     14
Property_Area         0
Loan_Status           0
Gender                0
Dependents            0
Self_Employed         0
Credit_History        1
dtype: int64

In [30]:
train.Credit_History.fillna(1,inplace=True)
train.isnull().sum()

Loan_ID               0
Married               3
Education             0
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount           22
Loan_Amount_Term     14
Property_Area         0
Loan_Status           0
Gender                0
Dependents            0
Self_Employed         0
Credit_History        0
dtype: int64

In [31]:
train.LoanAmount.median()

128.0

In [32]:
train.Loan_Amount_Term.median()

360.0

In [33]:
train.LoanAmount.replace({np.nan:125},inplace=True)
train.Loan_Amount_Term.replace({np.nan:360},inplace=True)
train.isnull().sum()

Loan_ID              0
Married              3
Education            0
ApplicantIncome      0
CoapplicantIncome    0
LoanAmount           0
Loan_Amount_Term     0
Property_Area        0
Loan_Status          0
Gender               0
Dependents           0
Self_Employed        0
Credit_History       0
dtype: int64

# filling the unknown values of married

In [34]:
train1=get_filled(train,'Married')

Loan_ID              0
Married              0
Education            0
ApplicantIncome      0
CoapplicantIncome    0
LoanAmount           0
Loan_Amount_Term     0
Property_Area        0
Loan_Status          0
Gender               0
Dependents           0
Self_Employed        0
Credit_History       0
dtype: int64


In [35]:
train1['Gender']=train1['Gender'].replace({1:'Male',0:'Female'})
train1['Dependents']=train1['Dependents'].replace({3:'3+',1:'1',2:'2'})
train1['Dependents']=train1['Dependents'].astype('object')
train1.dtypes

Loan_ID               object
Married               object
Education             object
ApplicantIncome        int64
CoapplicantIncome    float64
LoanAmount           float64
Loan_Amount_Term     float64
Property_Area         object
Loan_Status           object
Gender                object
Dependents            object
Self_Employed        float64
Credit_History       float64
dtype: object

In [36]:
train1['Self_Employed']=train1['Self_Employed'].replace({1:'Yes',0:'No'})
train1=clean(train1)
train1.isnull().sum()

Married           0
Education         0
Property_Area     0
Loan_Status       0
Gender            0
Dependents        0
Self_Employed     0
Credit_History    0
Income            0
loan              0
dtype: int64

In [37]:
train=new_df(df,train,'Married',train1)
train.isnull().sum()

Loan_ID              0
Education            0
ApplicantIncome      0
CoapplicantIncome    0
LoanAmount           0
Loan_Amount_Term     0
Property_Area        0
Loan_Status          0
Gender               0
Dependents           0
Self_Employed        0
Credit_History       0
Married              0
dtype: int64

In [38]:
train.head()

,Loan_ID,Education,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Property_Area,Loan_Status,Gender,Dependents,Self_Employed,Credit_History,Married
0,LP001002,Graduate,5849,0.0,125.0,360.0,Urban,Y,1,0,0.0,1.0,No
1,LP001003,Graduate,4583,1508.0,128.0,360.0,Rural,N,1,1,0.0,1.0,Yes
2,LP001005,Graduate,3000,0.0,66.0,360.0,Urban,Y,1,0,1.0,1.0,Yes
3,LP001006,Not Graduate,2583,2358.0,120.0,360.0,Urban,Y,1,0,0.0,1.0,Yes
4,LP001008,Graduate,6000,0.0,141.0,360.0,Urban,Y,1,0,0.0,1.0,No


In [39]:
train['Gender']=train['Gender'].replace({1:'Male',0:'Female'})
train['Dependents']=train['Dependents'].replace({3:'3+',1:'1',2:'2'})
train['Self_Employed']=train['Self_Employed'].replace({1:'Yes',0:'No'})
train=clean(train)

In [40]:
train.head()

,Education,Property_Area,Loan_Status,Gender,Dependents,Self_Employed,Credit_History,Married,Income,loan
0,1,3,Y,1,0,0,1.0,0,1,3
1,1,1,N,1,1,0,1.0,1,1,3
2,1,3,Y,1,0,1,1.0,1,0,3
3,0,3,Y,1,0,0,1.0,1,0,3
4,1,3,Y,1,0,0,1.0,0,1,4


In [41]:
train.Loan_Status=train0.Loan_Status

In [42]:
train=train[['Gender','Married','Dependents','Education','Self_Employed','Credit_History','Property_Area','Income','loan','Loan_Status']]
train.head()

,Gender,Married,Dependents,Education,Self_Employed,Credit_History,Property_Area,Income,loan,Loan_Status
0,1,0,0,1,0,1.0,3,1,3,Y
1,1,1,1,1,0,1.0,1,1,3,N
2,1,1,0,1,1,1.0,3,0,3,Y
3,1,1,0,0,0,1.0,3,0,3,Y
4,1,0,0,1,0,1.0,3,1,4,Y


In [43]:
train.to_csv('train_modified.csv',index=False)
#saving training data set